# Data Generation using LLM

In [2]:
from langchain.prompts import FewShotPromptTemplate,PromptTemplate
from langchain.chat_models import ChatOpenAI
import openai
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import numpy as np
from openai import OpenAI
import os

In [42]:
def get_completion(prompt: str, llm_model="gpt-4o"):
    openai.api_key = os.getenv("OPENAI_API_KEY")
    client = OpenAI(api_key=openai.api_key)
    messages = [{"role":"user","content":prompt}]
    response =client.chat.completions.create(model=llm_model,
                                           messages = messages,
                                           )
    return response.choices[0].message.content.strip()

In [5]:
def generate_text_data(data: pd.DataFrame, 
                       category:str,
                       use_samples: int,
                       number_of_tweets: int,
                      llm_model="gpt-4o-mini"):
    data = data[data["tweet"]!=""]
    cat_data = data[data["Disorder"]==category]
    cat_data.reset_index(drop=True,inplace=True)
    samples = np.random.choice(range(len(cat_data)),use_samples,replace=False)
    cat_data = cat_data.filter(items=samples, axis=0)
    tweets = cat_data["tweet"].values.tolist()
    tweets = "\n".join(tweets)
    prompt = f"""Given the following tweets:\n\n{tweets}\n\nGenerate `{number_of_tweets}` more unique tweets 
                using above tweets. Generated `{number_of_tweets}` tweets should be different from above tweets.
                `Do not add any extra text except tweets`"""
    prompt = prompt.replace("{", "{{").replace("}", "}}")
    prompt_template = PromptTemplate(input_variables=["tweet"], template=prompt)
    response = get_completion(prompt,llm_model=llm_model)
    return response

In [44]:
data_path = Path.cwd().parent/"Data"
final_data = pd.DataFrame()

In [45]:
for child in data_path.iterdir():
    for child_ch in child.iterdir():
        if child_ch.is_dir():
            files = list(child_ch.glob("**/*.csv"))
            for fls in tqdm(range(len(files))):
                df = pd.read_csv(files[fls])
        else:
            df = pd.read_csv(child_ch)
        final_data = pd.concat([df,final_data],axis = 0)

100%|██████████████████████████████████████████████████████████████████████| 24/24 [00:00<00:00, 94.45it/s]


In [1]:
final_data = final_data[["class","tweet"]]
final_data.rename(columns={"class":"Disorder"}, inplace=True)
final_data.head()

# Binary-class samples

In [9]:
binary_samples = (final_data["Disorder"].value_counts().sum()-final_data["Disorder"].value_counts()["CONTROL"]) - \
                                                        final_data["Disorder"].value_counts()["CONTROL"]
use_samples = 200
generate_samples = 100
for samples in tqdm(range(binary_samples//generate_samples)):
    generated_data = generate_text_data(final_data,"CONTROL",use_samples,generate_samples)
    generated_df = pd.DataFrame(data={"tweet":generated_data.split("\n")})
    generated_df["Disorder"]="CONTROL"
    final_data = pd.concat([final_data,generated_df], axis=0)
    final_data = final_data.sample(len(final_data))

In [15]:
final_data.Disorder.value_counts()

Disorder
ADHD               3034
SCHIZOPHRENIA      2970
OCD                2905
ANXIETY            2729
CONTROL            2631
PTSD               2466
DEPRESSION         2161
AUTISM             1425
EATING DISORDER     403
BIPOLAR             244
Name: count, dtype: int64

# Multi-class samples

In [2]:
final_data = x[x["Disorder"].isin(["AUTISM"])]
final_data.Disorder.value_counts()

In [119]:
num_samples = 5000
multi_class_samples = final_data["Disorder"].value_counts().to_dict()
out_of_band_samples = dict()
for cls,samples in multi_class_samples.items():
    if cls!="CONTROL":
        print(cls)
        out_of_band_samples.update({cls:num_samples-multi_class_samples[cls]})
        for samples in tqdm(range(out_of_band_samples[cls]//generate_samples)):
            generated_data = generate_text_data(final_data,cls,use_samples,generate_samples)
            generated_df = pd.DataFrame(data={"tweet":generated_data.split("\n")})
            generated_df["Disorder"]=cls
            final_data = pd.concat([final_data,generated_df], axis=0)
            final_data = final_data.sample(len(final_data)) 

AUTISM


100%|██████████████████████████████████████████████████████████████████████| 28/28 [09:46<00:00, 20.94s/it]


In [120]:
out_of_band_samples

{'AUTISM': 2840}

In [121]:
x = pd.read_csv("multi_class_data_balanced.csv")
final_data.reset_index(drop=True,inplace=True)
y = pd.concat([x,final_data], axis=0)
y = y.sample(len(y))